In [1]:
import sys
import gc
import os, shutil
import tempfile
from os import listdir
from random import randint

import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import imshow
import seaborn as sns
from PIL import Image

from keras import backend as K

import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from tensorflow.keras import models
from tensorflow.keras.preprocessing import image
from tensorflow.keras import mixed_precision, regularizers
from tensorflow.keras.metrics import top_k_categorical_accuracy
from tensorflow.keras.layers import Input, Add, Dropout, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.initializers import random_uniform, glorot_uniform, constant, identity, he_normal
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, CSVLogger
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.applications import InceptionV3, Xception, MobileNetV3Large,EfficientNetB0,EfficientNetV2B0
from resnet import resnet18

from sklearn.metrics import classification_report,confusion_matrix, matthews_corrcoef
from sklearn.utils.class_weight import compute_class_weight

In [2]:
plt.set_cmap('gray')
pd.set_option('precision', 3)

<Figure size 432x288 with 0 Axes>

In [3]:
for gpu in tf.config.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(gpu, True)

## Setting awal beberapa hyperparameter, serta penggunaan *mixed precision* dari NVIDIA CUDA/CU-DNN

In [4]:
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

def scheduler1(epoch, lr):
    if epoch < 6:
        return lr
    else:
        return lr * tf.math.exp(-0.05)

def scheduler2(epoch, lr):
    if epoch < 6:
        return lr
    else:
        return lr * tf.math.exp(-0.1)
    
def scheduler3(epoch, lr):
    if epoch < 6:
        return lr
    else:
        return lr * tf.math.exp(-0.15)
    
es = EarlyStopping(monitor='val_accuracy', mode='max', min_delta=0.005, patience = 6,  restore_best_weights=True)
lrs1 = LearningRateScheduler(scheduler1)
lrs2 = LearningRateScheduler(scheduler2)
lrs3 = LearningRateScheduler(scheduler3)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3060 Laptop GPU, compute capability 8.6


## Data Generator Utility

In [5]:
def create_datagen(train_path, val_path, test_path, target_size = (256,256), batch_size = 16, efficient = False):
    if efficient:
        train_datagen = image.ImageDataGenerator(
            rescale = 1.,
        )
    else:
        train_datagen = image.ImageDataGenerator(
            rescale = 1./255,
        )

    train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=target_size,
        batch_size= batch_size,
        color_mode="rgb",
        class_mode='categorical',
        shuffle = True
    )

    validation_generator = train_datagen.flow_from_directory(
        val_path,
        target_size=target_size,
        batch_size= batch_size,
        color_mode="rgb",
        class_mode='categorical',
        shuffle = False
    )
    
    test_generator = train_datagen.flow_from_directory(
        test_path,
        target_size=target_size,
        batch_size= batch_size,
        color_mode="rgb",
        class_mode='categorical',
        shuffle = False
    )
    return train_generator, validation_generator, test_generator

In [6]:
initializer = tf.keras.initializers.HeNormal()

def add_regularization(model, regularizer = regularizers.l2(0.01)):
    if not isinstance(regularizer, regularizers.Regularizer):
        print("Regularizer must be a subclass of tf.keras.regularizers.Regularizer")
        return model

    for layer in model.layers:
        for attr in ['kernel_regularizer']:
            if hasattr(layer, attr):
                setattr(layer, attr, regularizer)

    model_json = model.to_json()

    tmp_weights_path = os.path.join(tempfile.gettempdir(), 'tmp_weights.h5')
    model.save_weights(tmp_weights_path)

    model = models.model_from_json(model_json)
    
    model.load_weights(tmp_weights_path, by_name=True)
    return model

## Utility pembuat Model

In [7]:
def make_Inception():
    base = InceptionV3(
        include_top = False,
        weights = "imagenet",
        input_shape = (256, 256, 3),
        pooling = "max",
        classes = 4,
        classifier_activation="softmax",
    )
    out = Dropout(0.4)(base.output)
    out = Dense(32, activation='relu', kernel_initializer=initializer)(out)
    out = BatchNormalization()(out)
    out = Dropout(0.4)(out)
    out = Dense(4, activation='softmax', kernel_initializer="glorot_uniform")(out)

    model = Model(inputs = base.input,outputs=out)
    add_regularization(model)

    return model

In [8]:
def make_Xception():  
    base = Xception(
        include_top = False,
        weights = "imagenet",
        input_shape = (256, 256, 3),
        pooling = "max"
    )
    out = Dropout(0.5)(base.output)
    out = Dense(32, activation='relu', kernel_initializer=initializer)(out)
    out = BatchNormalization()(out)
    out = Dropout(0.5)(out)
    out = Dense(4, activation='softmax', kernel_initializer="glorot_uniform")(out)

    model = Model(inputs = base.input,outputs=out)
    add_regularization(model)
    
    return model

In [9]:
def make_mobileNet():
    base = MobileNetV3Large(
        input_shape= (256, 256, 3),
        alpha=1.0,
        include_top = False,
        weights= "imagenet",
        classes = 4,
        pooling = "max",
        dropout_rate = 0.3,
    )
    out = Dropout(0.5)(base.output)
    out = Dense(32, activation='relu', kernel_initializer=initializer)(out)
    out = BatchNormalization()(out)
    out = Dropout(0.5)(out)
    out = Dense(4, activation='softmax', kernel_initializer="glorot_uniform")(out)

    model = Model(inputs = base.input,outputs=out)

    return model

In [10]:
def make_EfficientB0():
    base = EfficientNetB0(
        include_top = False,
        weights = "imagenet",
        input_shape = (256, 256, 3),
        pooling = "max",
        classes = 4
    )
    out = Dropout(0.5)(base.output)
    out = Dense(64, activation='relu', kernel_initializer=initializer)(out)
    out = BatchNormalization()(out)
    out = Dropout(0.5)(out)
    out = Dense(4, activation='softmax', kernel_initializer="glorot_uniform")(out)

    model = Model(inputs = base.input,outputs=out)
    add_regularization(model)

    return model

## Define Path Dataset + Trial Generator

In [11]:
train_path = 'Categorized Datasets/Dataset/Distribution 1/Model D/train'
val_path = 'Categorized Datasets/Dataset/Distribution 1/Model D/val'
test_path = 'Categorized Datasets/Dataset/Distribution 1/Model D/test'

In [ ]:
train_generator, validation_generator, test_generator = create_datagen(train_path, val_path, test_path,
                                                                       batch_size = batch,efficient = False)

In [ ]:
x_batch, y_batch = next(train_generator)

plt.figure(figsize=(16, 32))
for k, (img, lbl) in enumerate(zip(x_batch, y_batch)):
    plt.subplot(8, 4, k+1)#4 rows with 8 images.
    plt.title(str(lbl))
    plt.imshow(img)

### Optimisasi untuk Model HQ,HPS,RSS - Inc, Xcp, Mbl, Eff - Random Search

In [12]:
best_hyperparam = (100, 0, 0)
epochs = 8

lr_trial = [1e-2, 1e-3, 1e-4, 5e-5]
batch_trial = [8, 16, 32]

trial = 10

#Round 1 Random Search (Coarse)
for _ in range(trial):
    lr = lr_trial[randint(0,3)]
    batch = batch_trial[randint(0,2)]
    print(lr, batch)
    train_generator, validation_generator, test_generator = create_datagen(train_path, val_path, test_path,
                                                                       batch_size = batch,efficient = True)
    model = make_EfficientB0()
    model.compile(loss = CategoricalCrossentropy(from_logits=False, label_smoothing = 0.2, axis=-1), 
                                optimizer = Adam(learning_rate=lr, beta_1=0.9, beta_2=0.999, 
                                            epsilon=None, amsgrad=False), 
                               metrics = ['accuracy'])
    hist = model.fit(
            train_generator,
            epochs = epochs,
            validation_data=validation_generator,
            verbose = 2
        )
    val_loss = np.nanmin(hist.history['val_loss'])
    print(f"Minimum val loss is {val_loss}")
    if val_loss < best_hyperparam[0]:
        best_hyperparam = (val_loss, lr, batch)
        print(best_hyperparam)
    del model
    gc.collect()
    tf.keras.backend.clear_session()

0.001 16
Found 3480 images belonging to 4 classes.
Found 101 images belonging to 4 classes.
Found 106 images belonging to 4 classes.
Instructions for updating:
Colocations handled automatically by placer.
Epoch 1/8


C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


218/218 - 55s - loss: 1.2595 - accuracy: 0.6632 - val_loss: 0.9249 - val_accuracy: 0.8911 - 55s/epoch - 251ms/step
Epoch 2/8
218/218 - 33s - loss: 0.8109 - accuracy: 0.9069 - val_loss: 0.9598 - val_accuracy: 0.8614 - 33s/epoch - 152ms/step
Epoch 3/8
218/218 - 34s - loss: 0.7202 - accuracy: 0.9589 - val_loss: 0.8160 - val_accuracy: 0.8713 - 34s/epoch - 155ms/step
Epoch 4/8
218/218 - 33s - loss: 0.6847 - accuracy: 0.9796 - val_loss: 0.8719 - val_accuracy: 0.8812 - 33s/epoch - 151ms/step
Epoch 5/8
218/218 - 33s - loss: 0.6686 - accuracy: 0.9851 - val_loss: 0.8016 - val_accuracy: 0.8713 - 33s/epoch - 152ms/step
Epoch 6/8
218/218 - 33s - loss: 0.6595 - accuracy: 0.9908 - val_loss: 0.8103 - val_accuracy: 0.9109 - 33s/epoch - 152ms/step
Epoch 7/8
218/218 - 33s - loss: 0.6483 - accuracy: 0.9937 - val_loss: 0.8758 - val_accuracy: 0.8812 - 33s/epoch - 153ms/step
Epoch 8/8
218/218 - 33s - loss: 0.6495 - accuracy: 0.9920 - val_loss: 0.8880 - val_accuracy: 0.8416 - 33s/epoch - 152ms/step
Minimum va

Epoch 1/8
435/435 - 78s - loss: 2.1460 - accuracy: 0.3023 - val_loss: 1.2524 - val_accuracy: 0.4752 - 78s/epoch - 179ms/step
Epoch 2/8
435/435 - 62s - loss: 1.8766 - accuracy: 0.3753 - val_loss: 1.0369 - val_accuracy: 0.6535 - 62s/epoch - 142ms/step
Epoch 3/8
435/435 - 62s - loss: 1.6790 - accuracy: 0.4397 - val_loss: 0.9412 - val_accuracy: 0.7426 - 62s/epoch - 142ms/step
Epoch 4/8
435/435 - 61s - loss: 1.4879 - accuracy: 0.5037 - val_loss: 0.8987 - val_accuracy: 0.8317 - 61s/epoch - 141ms/step
Epoch 5/8
435/435 - 61s - loss: 1.3494 - accuracy: 0.5805 - val_loss: 0.8825 - val_accuracy: 0.8515 - 61s/epoch - 141ms/step
Epoch 6/8
435/435 - 62s - loss: 1.2432 - accuracy: 0.6333 - val_loss: 0.8653 - val_accuracy: 0.8515 - 62s/epoch - 141ms/step
Epoch 7/8
435/435 - 62s - loss: 1.1676 - accuracy: 0.6770 - val_loss: 0.8702 - val_accuracy: 0.8515 - 62s/epoch - 142ms/step
Epoch 8/8
435/435 - 62s - loss: 1.1160 - accuracy: 0.7190 - val_loss: 0.8617 - val_accuracy: 0.8515 - 62s/epoch - 142ms/step


In [13]:
print(f"START RANDOM SEARCH ROUND 2 (FINE)")

epochs = 5
train_generator, validation_generator, test_generator = create_datagen(train_path, val_path, test_path,
                                                                       batch_size = best_hyperparam[2],efficient = False)

#Round 2 Random Search (Fine)
lr_trial_2 = [ best_hyperparam[1] / 2, best_hyperparam[1] * 2 ]
for lr in lr_trial_2:
    model = make_EfficientB0()
    model.compile(loss = CategoricalCrossentropy(from_logits=False, label_smoothing = 0.2, axis=-1), 
                                optimizer = Adam(learning_rate=lr, beta_1=0.9, beta_2=0.999, 
                                            epsilon=None, amsgrad=False), 
                               metrics = ['accuracy'])
    hist = model.fit(
            train_generator,
            epochs = epochs,
            validation_data=validation_generator,
            verbose = 2
        )
    val_loss = np.nanmin(hist.history['val_loss'])
    print(f"Minimum val loss is {val_loss}")
    if val_loss < best_hyperparam[0]:
        best_hyperparam = (val_loss, lr, batch)
        print(best_hyperparam)
    del model
    gc.collect()
    tf.keras.backend.clear_session()

print(f"START RANDOM SEARCH ROUND 3 (VERY FINE)")
#Round 3 Random Search (Very Fine)
lr_trial_3 = [ best_hyperparam[1] / 3, best_hyperparam[1] * 2/3 ,  
              best_hyperparam[1] * 4/3, best_hyperparam[1] * 5/3]
for lr in lr_trial_3:
    model = make_mobileNet()
    model.compile(loss = CategoricalCrossentropy(from_logits=False, label_smoothing = 0.2, axis=-1), 
                                optimizer = Adam(learning_rate=lr, beta_1=0.9, beta_2=0.999, 
                                            epsilon=None, amsgrad=False), 
                               metrics = ['accuracy'])
    hist = model.fit(
            train_generator,
            epochs = epochs,
            validation_data=validation_generator,
            verbose = 2
        )
    val_loss = np.nanmin(hist.history['val_loss'])
    print(f"Minimum val loss is {val_loss}")
    if val_loss < best_hyperparam[0]:
        best_hyperparam = (val_loss, lr, batch)
        print(best_hyperparam)
    del model
    gc.collect()
    tf.keras.backend.clear_session()

START RANDOM SEARCH ROUND 2 (FINE)
Found 3480 images belonging to 4 classes.
Found 101 images belonging to 4 classes.
Found 106 images belonging to 4 classes.
Epoch 1/5
218/218 - 50s - loss: 1.4721 - accuracy: 0.5652 - val_loss: 1.5719 - val_accuracy: 0.2178 - 50s/epoch - 232ms/step
Epoch 2/5
218/218 - 34s - loss: 0.9715 - accuracy: 0.8193 - val_loss: 1.3973 - val_accuracy: 0.2970 - 34s/epoch - 157ms/step
Epoch 3/5
218/218 - 34s - loss: 0.8279 - accuracy: 0.9060 - val_loss: 1.0928 - val_accuracy: 0.7525 - 34s/epoch - 157ms/step
Epoch 4/5
218/218 - 34s - loss: 0.7726 - accuracy: 0.9362 - val_loss: 1.0791 - val_accuracy: 0.7624 - 34s/epoch - 157ms/step
Epoch 5/5
218/218 - 34s - loss: 0.7245 - accuracy: 0.9618 - val_loss: 1.1465 - val_accuracy: 0.6634 - 34s/epoch - 158ms/step
Minimum val loss is 1.0791015625
Epoch 1/5
218/218 - 50s - loss: 1.3087 - accuracy: 0.6236 - val_loss: 2.0967 - val_accuracy: 0.2079 - 50s/epoch - 230ms/step
Epoch 2/5
218/218 - 34s - loss: 0.8060 - accuracy: 0.8951 

In [14]:
best_hyperparam

(0.7833426339285714, 0.001, 16)

In [ ]:
gc.collect()
tf.keras.backend.clear_session()